In [2]:
# 参考：http://akiyoko.hatenablog.jp/entry/2015/01/04/114642
# 置換対象クラス
"""
############
# models.py#
############
import random

class User(object):
    def __init__(self, name, gender='m'):
        self.name = name
        self.gender = gender

    def get_name(self):
        return self.name

    def get_gender(self):
        return self.gender

    def vote(self, *seq):
        return random.choice(seq)

    @classmethod
    def class_method(cls):
        return 'class method'

    @staticmethod
    def static_method():
        return 'static method'
"""

In [3]:
###########################################
#          直接mockで置き換える           #
###########################################
import models
from unittest import mock

In [4]:
### get_nameメソッドをMagicMockオブジェクトで直接置き換える
user = models.User("midorikwa")
user.get_name = mock.MagicMock()
user.get_name.return_value = "mock"
# user.get_name = mock.MagicMock(return_value="mock")としてもOK
print(user.get_name())

mock


In [5]:
### UserクラスをMagicMockオブジェクトで直接置き換える
models.User = mock.MagicMock()
models.User.return_value.get_name.return_value = "mock"
user = models.User("midorikawa")
print(user.get_name)

<MagicMock name='mock().get_name' id='139830785833672'>


In [1]:
###########################################
#           クラスを置き換える            #
###########################################
import models
from unittest import mock

# この章を始めるときは一度ipython notebookを再起動しましょう

In [9]:
# patch(target) as ... で後から振る舞いを規定する
# =================================================
"""
patch でモック化したオブジェクトは as で受け取ることができるので、
受け取ったモックオブジェクトに対して後から振る舞いを規定することができる。

※第一引数（target）は、、、
1. import できる形の文字列であること
2. クラス名、メソッド名、関数名のいずれかであること(モジュール名は不可)
"""

'\npatch でモック化したオブジェクトは as で受け取ることができるので、\n受け取ったモックオブジェクトに対して後から振る舞いを規定することができる。\n\n※第一引数（target）は、、、\n1. import できる形の文字列であること\n2. クラス名、メソッド名、関数名のいずれかであること(モジュール名は不可)\n'

In [4]:
#■with構文
with mock.patch("models.User") as mock_user:
    mock_user.return_value.get_name.return_value = "mock"
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [5]:
#■デコレータ
@mock.patch("models.User")
def test_user(mock_user):
    mock_user.return_value.get_name.return_value = "mock"
    user = models.User()
    print(user.get_name())
test_user()

mock


In [6]:
#■置換対象のモジュールをfrom hoge import hogehogeとインポートしている場合
from models import User
with mock.patch("__main__.User") as mock_user:
    mock_user.return_value.get_name.return_value = "mock"
    user = User("midorikawa")
    print(user.get_name)

<MagicMock name='User().get_name' id='140121700246752'>


In [1]:
###########################################
#        メソッドを置き換える             #
###########################################
import models
from unittest import mock

# この章を始めるときは一度ipython notebookを再起動しましょう

In [2]:
# patch(target, return_value=...) を使う
# =================================================
"""
return_value で、メソッドが実行されたときの戻り値を規定することができる。
"""

'\nreturn_value で、メソッドが実行されたときの戻り値を規定することができる。\n'

In [3]:
#■patch, with構文
with mock.patch("models.User.get_name", return_value="mock") as _mock_get_name:
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [4]:
#■patch.object, with構文
with mock.patch.object(models.User, "get_name", return_value="mock") as _mock_get_name:
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [5]:
#■patch, デコレータ
@mock.patch("models.User.get_name", return_value="mock")
def test_user(_mock_get_name):
    user = models.User("midorikawa")
    print(user.get_name())
test_user()

mock


In [6]:
#■patch.object, デコレータ
@mock.patch.object(models.User, "get_name", return_value="mock")
def test_user(_mock_get_name):
    user = models.User("midorikawa")
    print(user.get_name())
test_user()

mock


In [7]:
# patch(target, side_effect=...) を使う
# =================================================
"""
side_effectは、メソッドの定義そのものの書き換え、
エラー出力、イテレータで順に戻り値を指定することができる。
"""

'\nside_effectは、メソッドの定義そのものの書き換え、\nエラー出力、イテレータで順に戻り値を指定することができる。\n'

In [8]:
#■メソッドの定義そのものの書き換え
def mock_get_name():
    return "mock"

with mock.patch("models.User.get_name", side_effect=mock_get_name) as _mock_get_name:
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [10]:
#■エラー出力
with mock.patch("models.User.get_name", side_effect=Exception("hello")) as _mock_get_name:
    user = models.User("midorikawa")
    user.get_name()

Exception: hello

In [12]:
#■イテレータで順に戻り値を指定
with mock.patch("models.User.get_name", side_effect=["spam", "ham", "egg"]) as _mock_get_name:
    user = models.User("midorikawa")
    for i in range(4):
        print(user.get_name())

spam
ham
egg


StopIteration: 

In [13]:
# patch(target, new) を使う
# =================================================
"""
patch の第2引数、および patch.object の第3引数に、置き換えるモックオブジェクトを指定することができる。
ただし、指定できるのはモックオブジェクトに限る。
"""

'\npatch の第2引数、および patch.object の第3引数に、置き換えるモックオブジェクトを指定することができる。\nただし、指定できるのはモックオブジェクトに限る。\n'

In [16]:
mock_get_name = mock.MagicMock(return_value="mock")
with mock.patch("models.User.get_name", mock_get_name) as _mock_get_name:
    user = models.User("midorikawa")
    print(user.get_name())
    
#　↑と↓は同義

with mock.patch("models.User.get_name", mock.MagicMock(return_value="mock")) as _mock_get_name:
    user = models.User("midorikawa")
    print(user.get_name())

mock
mock


In [17]:
# patch(target) as ... で後から振る舞いを規定する
# =================================================
"""
patch でモック化したオブジェクトは as で（デコレータを使った場合はメソッドの引数として）受け取ることができるので、
受け取ったモックオブジェクトに対して後から振る舞いを規定することができる。
"""

'\npatch でモック化したオブジェクトは as で（デコレータを使った場合はメソッドの引数として）受け取ることができるので、\n受け取ったモックオブジェクトに対して後から振る舞いを規定することができる。\n'

In [18]:
with mock.patch("models.User.get_name") as _mock_get_name:
    _mock_get_name.return_value = "mock"
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [20]:
with mock.patch("models.User.get_name") as _mock_get_name:
    _mock_get_name.side_effect = ["spam", "ham", "egg"]
    user = models.User("midorikawa")
    for i in range(3):
        print(user.get_name())

spam
ham
egg


In [21]:
###########################################
#           関数を置き換える             #
###########################################
import models
from unittest import mock

# この章を始めるときは一度ipython notebookを再起動しましょう

In [22]:
#■関数をmockに置き換える(with構文)
with mock.patch("random.choice", return_value=0) as _mock_choice:
    user = models.User("midorikawa")
    print(user.vote(1,2,3))

0


In [27]:
#■関数をmockに置き換える(デコレータ)
@mock.patch("random.choice", return_value=0)
def test_user(_mock_choice):
    user = models.User("midorikawa")
    print(user.vote(1,2,3))
test_user()

0


In [29]:
#■組み込み関数を置き換える
with mock.patch("builtins.input", return_value="hello") as _mock_input:
    ret = input("input : ")
    print(ret)

with mock.patch("__main__.input", return_value="hello") as _mock_input:
    ret = input("input : ")
    print(ret)

@mock.patch("builtins.input", return_value="hello")
def test_input(_mock_input):
    ret = input("input : ")
    print(ret)
test_input()

@mock.patch("__main__.input", return_value="hello")
def test_input(_mock_input):
    ret = input("input : ")
    print(ret)
test_input()

hello
hello
hello
hello


In [30]:
######################################################################################
#      （クラスがインスタンス化された際の）インスタンスオブジェクトを置き換える      #
######################################################################################
import models
from unittest import mock

# この章を始めるときは一度ipython notebookを再起動しましょう

In [31]:
# patch(クラス名, return_value=...) を使う
# =================================================
"""
patch(クラス名, return_value=...) を使うと、クラスがインスタンス化された際のインスタンスオブジェクトを置き換えることができる。
"""

'\npatch(クラス名, return_value=...) を使うと、クラスがインスタンス化された際のインスタンスオブジェクトを置き換えることができる。\n'

In [34]:
#■patch, with
mock_user = mock.MagicMock()
mock_user.get_name.return_value = 'mock'
with mock.patch("models.User", return_value=mock_user) as _mock_user:
    user = models.User("midorikawa")
    print(user.get_name())

mock


In [36]:
#■patch, デコレータ
mock_user = mock.MagicMock()
mock_user.get_name.return_value = "mock"

@mock.patch("models.User", return_value=mock_user)
def test_user(_mock_user):
    user = models.User("midorikawa")
    print(user.get_name())
test_user()

mock
